**AUGMENTATION SCRIPT ON COLAB**

1. Upload the augmentation script into your Google Drive   
2. Mount your drive & navigate to your augmentation script folder
3. Replace your Roboflow dataset API (if needed)
4. Configure the augmentation modules as per your need
5. Adjust the augmentation parameters accordingly

In [1]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Navigate to script location
%cd "/content/drive/MyDrive/vision_ai/for colab"

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Mounted at /content/drive
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1029664936.py", line 6, in <cell line: 0>
    get_ipython().run_line_magic('cd', '"/content/drive/MyDrive/vision_ai/for_colab"')
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
                              ^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
             ^^^^^^^^^^^
OSError: [Errno 107] Transport endpoint is not connected

During handling of the ab

In [2]:
# Install libraries that do not available in Colab
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.6/88.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 89.6 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


In [3]:
# Configure the augmentation modules as per your need
# Mainly augmentation done on train dataset
def get_train_transforms():
    transform = A.ReplayCompose(
        [
            # A.Resize(1024, 1024, p=1.0),
            A.LongestMaxSize(max_size=1024, interpolation=cv2.INTER_LINEAR, p=1.0),
            ########################################## random scale implementation of object-of-interests ##################################################
            # Additional: Random scale and padded to adapt various scales of object of interest
            A.RandomScale(scale_limit=(-0.5, -0.8),interpolation=cv2.INTER_LINEAR,mask_interpolation=cv2.INTER_NEAREST,area_for_downscale="image", p=0.4),
            # A.PadIfNeeded(min_height=1024, min_width=1024, border_mode=cv2.BORDER_CONSTANT, fill=0, fill_mask=0, p=1.0),
            ################################################################################################################################################
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.3),
            A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, p=0.4),
            A.ToGray(p=0.2),
            A.HueSaturationValue(hue_shift_limit=15, sat_shift_limit=25, p=0.3),
            A.RandomBrightnessContrast(p=0.4),
            A.MotionBlur(blur_limit=(3, 5), p=0.3),
            A.Emboss(alpha=(0.2, 0.5), strength=(0.2, 0.7), p=0.5),
            A.CLAHE(clip_limit=(1, 4), tile_grid_size=(8, 8), p=0.8),
            A.ElasticTransform(alpha=100, sigma=50, p=0.5),
            A.Downscale(scale_range=(0.5, 0.75), interpolation_pair={'downscale': cv2.INTER_NEAREST, 'upscale': cv2.INTER_LINEAR}, p=0.5),
        ],
        bbox_params=A.BboxParams(
            format="coco",
            label_fields=["category_ids"]
        )
    )
    return transform

In [4]:
# No augmentation for valid and test dataset (just standard resize)
def get_valid_transforms():
    transform = A.ReplayCompose(
        [
            # A.Resize(1024, 1024, p=1.0),
            A.LongestMaxSize(max_size=1024, interpolation=cv2.INTER_LINEAR, p=1.0),
        ],
        bbox_params=A.BboxParams(
            format="coco",
            label_fields=["category_ids"]
        )
    )
    return transform

In [5]:
# Import on-premise libraries
import os
import cv2
import augmenting_coco_1 as ag            # your augmentation script name
import albumentations as A

# Replace your Roboflow dataset API
def roboflow_dataset():
  from roboflow import Roboflow
  rf = Roboflow(api_key="Mnx4uGjLhe58M3vt56Xi")
  project = rf.workspace("irnow-1").project("lead_frame")
  version = project.version(8)
  dataset = version.download("coco-segmentation")


  return dataset

ModuleNotFoundError: No module named 'augmenting_coco_1'

In [ ]:
PROJECT_NAME = 'Lead_Frame'

# Adjust your augmentation parameters and run the script
ag.main(
    data_type       = 'roboflow',                                                               # 'local' (local drive) and 'roboflow'
    local_data      = None,
    roboflow_data   = roboflow_dataset().location,                                              # roboflow_dataset().location or None
    result_path     = os.path.join('/content/drive/MyDrive/training_data', PROJECT_NAME),
    data_dist       = {'train': 3, 'valid': 1, 'test': 1},
    train_transform = get_train_transforms(),
    valid_transform = get_valid_transforms(),
)